# Detecting seasonal effects and trends, calculating moving average and other window statistics

## Settings

In [ ]:
import sys
import glob
import os
import pickle
import statistics
import math
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot, lag_plot, register_matplotlib_converters
register_matplotlib_converters()
from pandas.tseries.offsets import *

import seaborn as sns

import calendar
calendar.setfirstweekday(calendar.MONDAY) # first week day

from datetime import datetime, time, date, timedelta

from dateutil.relativedelta import *

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import MO, TU, WE, TH, FR, SA
from matplotlib.ticker import FuncFormatter
%matplotlib inline

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# pandas settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [ ]:
def plotseasonal(res, axes):
    res.observed.plot(ax=axes[0], legend=False)
    axes[0].set_ylabel('Observed')
    res.trend.plot(ax=axes[1], legend=False)
    axes[1].set_ylabel('Trend')
    res.seasonal.plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Seasonal')
    res.resid.plot(ax=axes[3], legend=False)
    axes[3].set_ylabel('Residual')

In [ ]:
def get_week_of_month(year, month, day):
    x = np.array(calendar.monthcalendar(year, month))
    week_of_month = np.where(x == day)[0][0] + 1
    return(week_of_month)

## Import the data

In [ ]:
data_path = ''

In [ ]:
# data source 1...
file = os.path.join(data_path, '')
data = pd.read_excel(file, header = [0, 1])

# data source 2...
file_2 = os.path.join(data_path, '')
data_new = pd.read_csv(file_2)

In [ ]:
data.head()

In [ ]:
# Renaming columns
data.columns = ['', '']

In [ ]:
# fixing date format
data['Datum'] = data['Datum'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
# One-hot-encoding
data = pd.get_dummies(data, columns=[''], drop_first=False)
corr = data_for_exploration.corr()

### Merging data (append or join)

In [ ]:
# Fixing index and sort
data.set_index(pd.DatetimeIndex(data['Datum']), inplace = True, drop = False)
data_new.set_index(pd.DatetimeIndex(data_new['Datum']), inplace = True, drop = False)
data.sort_index(inplace=True)
data = data.append(data_new, sort = True)

In [ ]:
data = pd.merge(data, data_new, how = 'outer', left_index = True, right_index = True)

## Exploration

In [ ]:
fig = plt.figure(figsize=(18, 4))

x = data['Datum']

plt.plot(x, data[''], label = '')
plt.plot(x, data[''], label='')
# Vertical lines, e.g. to show the days of ice jams
plt.vlines(data_new['Datum'], ymin, ymax, color = 'orange')
plt.vlines(data_new['Datum'], ymin, ymax, color = 'green')

#If some period is needed:
#ymin, ymax = plt.ylim()
#plt.xlim('2019-02', '2019-08')

plt.ylabel('N')
plt.legend()
plt.show()

### Seasonality and trends

In [ ]:
# To analyze seasonality: fill missing days with NANs (wokrdays in that case!)
data.reindex(pd.date_range('2015-01-01', '2020-01-30', freq = BDay()))

In [ ]:
res_1_ant = seasonal_decompose(data.iloc[2:][''], freq = 7) #week 
res_2_ant = seasonal_decompose(data[''], freq = 30)
res_3_ant = seasonal_decompose(data.iloc[:-22][''], freq = 365)

##### Week

In [ ]:
# Plot with only weekdays
res_1_ant.seasonal.plot(figsize = (20, 5))

ax = plt.gca()
plt.gca().set_xlim([date(2015, 1, 3), date(2015, 5, 28)])
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m (%a)'))
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=(MO, TU, WE, TH, FR)))
plt.xticks(rotation=70)
plt.show()

In [ ]:
# boxplots
sns.set(rc={'figure.figsize':(8, 4)})
sns.boxplot(data = data, y = data[''], x = data['Tag'], 
            order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], 
            palette = 'muted', showfliers = False)

##### Month

In [ ]:
res_2_ant.seasonal.plot(figsize = (20, 5))

ax = plt.gca()
plt.gca().set_xlim([date(2015, 1, 3), date(2016, 5, 28)])
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y'))
plt.xticks(rotation=70)
plt.show()

##### Year

In [ ]:
res_3_ant.seasonal.plot(figsize = (20, 5))

ax = plt.gca()
plt.gca().set_xlim([date(2015, 1, 3), date(2016, 5, 28)])
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y'))
plt.xticks(rotation=70)
plt.show()

#### Week+Month+Year

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=4, sharex=True, figsize=(19, 5))

plotseasonal(res_1_ant, axes[:,0])
plotseasonal(res_2_ant, axes[:,1])
plotseasonal(res_3_ant, axes[:,2])

plt.tight_layout()
plt.show()

### Correlations (including autocorrelation)

In [ ]:
plt.figure(figsize=(17, 5))
autocorrelation_plot(data[''])
plt.ylim(-0.2, 0.5)
plt.show()

In [ ]:
# The last 252 days closer
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
fig = plot_acf(data[''], lags = 260, ax = ax)
plt.ylim(-0.1, 0.4)
plt.show()

In [ ]:
data[''].corr(data[''])

### Lag-features and other time-related features

In [ ]:
# Avoid information leakage (by prediction for 3 days ahead, lag feature t-2 cannot be used)
for i in range(1, 15):
    data[' t-{}'.format(i)] = data_for_exploration[''].shift(i)
for i in range(1, 15):
    data[' t-{}'.format(i)] = data[''].shift(i)
    
# Future lags
data[' t+{}'] = data[''].shift(1)
data[' t+{}'] = data[''].shift(2)

In [ ]:
# Aggregate by week
#anfrage_antrag_w = data.resample('W-FRI').agg({
data_w = data.resample('W').agg({
    '': np.sum, 
    '': 'max'
})

In [ ]:
# Add month and year as columns, enumarate weeks within year/month
data['Month'] = data.index.month
data['Year'] = data.index.year
data['Day'] = data.index.day
data['Week_number_within_year'] = data.index.week
# Enumerate weeks within month
data['Week_number_within_month'] =\
       data.apply(lambda row: get_week_of_month(int(row['Year']), int(row['Month']), int(row['Day'])), axis = 1)
data = data.drop(['Day'], 1)
data['Year + month'] = pd.to_datetime(data.index).strftime("%Y-%m")

In [ ]:
# Window statistics (past)
for width in (4, 8, 12, 16, 20, 24):
    shifted_dataframe = data.shift(3) # Fix if no information leakage
    data['_sum_window-{}'.format(width)] = shifted_dataframe[''].rolling(window = width).sum()
    data['_mean_window-{}'.format(width)] = shifted_dataframe[''].rolling(window = width).mean()
# Window for the future
for width in (4, 8, 12, 16, 20, 24):
    data['_sum_window+{}'.format(width)] = data[''].rolling(window = width).sum().shift(-1*width)
    data['_mean_window+{}'.format(width)] = data[''].rolling(window = width).mean().shift(-1*width)

In [ ]:
# The last days closer
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
fig = plot_acf(data[''], lags = 60, ax = ax)
plt.ylim(-0.2, 0.6)
plt.show()

In [ ]:
# Moving average
def plotMovingAverage(series, window, plot_intervals=False, scale=1.96, plot_anomalies=False):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 

    """
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")
        
        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            plt.plot(anomalies, "ro", markersize=10)
        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.xticks(rotation = 70)
    plt.grid(True)

In [ ]:
plotMovingAverage(data[''], 52, plot_intervals=False, scale=1.96, plot_anomalies=False)

## Saving as pickle

In [ ]:
data.fillna(0, inplace = True)

In [ ]:
file_name = 'data.pkl'
file = os.path.join(data_path, file_name)
data.to_pickle(file)